In [ ]:
# Importing all necessary libraries 
import cv2 
import os 
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
import pandas as pd
import numpy as np
import sys
from scipy.spatial import distance
import json

metric = 'cosine'

model_url = "https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2"

IMAGE_SHAPE = (224, 224)

layer = hub.KerasLayer(model_url)
model = tf.keras.Sequential([layer])




In [ ]:
# Read the video from specified path and convert it into 10 FPS each video
def convert_to_frames(filename):
  os.system("ffmpeg -i {0} -filter:v fps=10 {1}".format(filename,filename.split('.')[0]+"_changed.mp4"))
  filename = filename.split(".")[0]
  cam = cv2.VideoCapture("{}_changed.mp4".format(filename)) 
  cam.set(cv2.CAP_PROP_FPS, 10) 
  try: 

    # creating a folder named data 
    if not os.path.exists('data/{}'.format(filename)): 
      os.makedirs('data/{}'.format(filename)) 

  # if not created then raise error 
  except OSError: 
    print ('Error: Creating directory of data') 

  # frame 
  currentframe = 0

  while(True): 
    
    # reading from frame 
    ret,frame = cam.read() 

    if ret: 
      # if video is still left continue creating images 
      name = './data/{}/frame'.format(filename) + str(currentframe) + '.jpg'
      print ('Creating...' + name)

      # writing the extracted images 
      cv2.imwrite(name, frame) 

      # increasing counter so that it will 
      # show how many frames are created 
      currentframe += 1
    else: 
      break

  # Release all space and windows once done 
  cam.release() 
  cv2.destroyAllWindows() 

In [ ]:
def extract(file):
  file = Image.open(file).convert('L').resize(IMAGE_SHAPE)
  #display(file)
  file = np.stack((file,)*3, axis=-1)
  file = np.array(file)/255.0
  embedding = model.predict(file[np.newaxis, ...])
  vgg16_feature_np = np.array(embedding)
  flattended_feature = vgg16_feature_np.flatten()
  return flattended_feature

In [ ]:
def convert_to_vectors(filename):
  filename = filename.split(".")[0]
  video_1_temp = os.listdir('data/{}/'.format(filename))
  video_1 = []
  for i in video_1_temp:
    video_1.append(extract('data/{}/'.format(filename)+i))
  return video_1  

In [ ]:
def find_minimum_distance_with_image(video_dic,image_to_find,threshold):
  dist_dic = {}
  for vd in video_dic:
    video_1 = video_dic[vd]
    temp_video_1 = []
    for i in video_1:
      temp_video_1.append(distance.cdist([i], [image_to_find], metric)[0][0])
    video_1_min = min(temp_video_1)
    dist_dic[vd] = video_1_min
  for i in dist_dic:
    if dist_dic[i] <= threshold:
      return i
    

In [ ]:
def find_minimum_distance_with_video(video_dic,video_to_find):
  dist_dic = {}
  for vd in video_dic:
    video_1 = video_dic[vd]
    mini = 1
    i = 0
    while i < len(video_1):
      temp_video_1 = []
      for j in video_to_find:
        temp_video_1.append(distance.cdist([video_1[i]], [j], metric)[0][0])
      mini = min(mini,min(temp_video_1))
      i+=len(video_1)
    dist_dic[vd] = mini
  for i in dist_dic:
    if dist_dic[i] <= threshold:
      return i


In [ ]:
video_1 = "1.mp4"
video_2 = "2.mp4"
video_3 = "3.mp4"
threshold = 0.1
convert_to_frames(video_1)
convert_to_frames(video_2)
convert_to_frames(video_3)
video_dic = {video_1:convert_to_vectors(video_1),video_2:convert_to_vectors(video_2),video_3:convert_to_vectors(video_3)}


In [ ]:
import pickle 
filehandler = open('vectors.obj','wb')
pickle.dump(filehandler,video_dic)



In [ ]:
print(video_dic)

In [ ]:
df = pd.DataFrame({"Name":[video_1,video_2,video_3],"Vectors":[video_dic[video_1],video_dic[video_2],video_dic[video_3]]})
df.to_csv("Video_data_vectors.csv")

In [ ]:
def get_video_metadata(image_to_find):
  threshold = 0.5
  image_to_find = extract(image_to_find)
  filehandler = open('vectors.obj','rb')
  temp = pickle.load(filehandler)
  # prin?t(temp)
  print(find_minimum_distance_with_image(temp,image_to_find,threshold))

In [ ]:
get_video_metadata("/content/drive/MyDrive/project/pigeon.jpeg")                                  

3.mp4
